# Install Requirements

In [ ]:
# clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo

%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
import pandas as pd
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

/content/drive/MyDrive/vehicle_detection/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.5 MB/s eta 0:00:00
Setup complete. Using torch 1.13.1+cu116 (Tesla T4)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r ../drive/MyDrive/vehicle_detection/yolo_dataset ..
!cp -r ../drive/MyDrive/vehicle_detection/vd.yaml ..
!cp -r ../drive/MyDrive/vehicle_detection/vedai.yaml ..

Mounted at /content/drive
cp: cannot stat '../drive/MyDrive/vehicle_detection/yolo_dataset': No such file or directory
cp: cannot stat '../drive/MyDrive/vehicle_detection/vd.yaml': No such file or directory


In [ ]:
!pip install wandb &> /dev/null
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Train Our Custom YOLOv5 model

In [ ]:
!python train.py --img 1280 --batch 4 --epochs 600 --data ./vd.yaml --hyp hyp.scratch-med.yaml --weights yolov5l.pt --cache --patience 50

wandb: Currently logged in as: carlolonghi. Use `wandb login --relogin` to force relogin
train: weights=yolov5l.pt, cfg=, data=./data/vd.yaml, hyp=hyp.scratch-med.yaml, epochs=600, batch_size=4, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=50, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch origin' timed out after 5 seconds
YOLOv5 🚀 v7.0-59-gfdc35b1 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.3, cls_pw=1.0, obj=0.7, obj_pw=1.0, iou_t=

# Pretrain on Vedai

In [ ]:
import shutil

def remove_suffix(input_string, suffix):
    if suffix and input_string.endswith(suffix):
        return input_string[:-len(suffix)]
    return input_string

if not os.path.exists('../yolo_vedai'):
    os.makedirs('../yolo_vedai')
    for dir in ('images', 'labels'):
        os.makedirs(os.path.join('../yolo_vedai', dir))
        for split in ('train', 'val'):
            os.makedirs(os.path.join('../yolo_vedai', dir, split))

for split in ('train', 'val'):
    df = pd.read_csv('../vedai_dataset/'+split+'_labels.csv', index_col=0)

    for img_file in df['file'].unique():
        shutil.copy(os.path.join('../vedai_dataset/images', img_file), os.path.join('../yolo_vedai/images', split, img_file))
        ann_file = remove_suffix(img_file, 'jpg') + 'txt'
        shutil.copy(os.path.join('../vedai_dataset/annotations', ann_file), os.path.join('../yolo_vedai/labels', split, ann_file))

In [ ]:
!python train.py --img 832 --batch 8 --epochs 200 --data ./vedai.yaml --hyp hyp.scratch-med.yaml --weights yolov5l.pt --cache --patience 15

wandb: Currently logged in as: carlolonghi. Use `wandb login --relogin` to force relogin
train: weights=yolov5l.pt, cfg=, data=./data/vedai.yaml, hyp=hyp.scratch-med.yaml, epochs=200, batch_size=8, imgsz=832, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=15, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 13 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v7.0-59-gfdc35b1 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, wa

# Finetune

In [ ]:
!python train.py --img 1280 --batch 4 --epochs 600 --data ./vd.yaml --hyp hyp.scratch-med.yaml --weights runs/train/exp3/weights/best.pt --cache --patience 200

wandb: Currently logged in as: carlolonghi. Use `wandb login --relogin` to force relogin
train: weights=runs/train/exp3/weights/best.pt, cfg=, data=./data/vd.yaml, hyp=hyp.scratch-med.yaml, epochs=600, batch_size=4, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=200, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 15 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v7.0-59-gfdc35b1 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, war